# Task 1

In [1]:
%matplotlib nbagg

In [2]:
import numpy as np
import strid
import matplotlib.pyplot as plt
import scipy.signal
import geneate_vibration_data

/Users/emilneby/miniforge3/envs/tkt4550/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Generate random vibration data for a shear frame of 8 storeys

In [3]:
geneate_vibration_data.generate_data(8, "results/data-stochastic_8_floor.npz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Use covariance-driven stochastic subspace identification to perform system identification

In [4]:
import covSSI

ssid, modes = covSSI.covSSI("results/data-stochastic_8_floor.npz")

In [5]:
%matplotlib nbagg
stabdiag = strid.StabilizationDiagram()
stabdiag.plot(modes)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>

# Task 2
### Task 2.1 - finding relative difference

In [6]:
cluster_meat = []
num_modes = 0

for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))
    num_modes += len(modes_of_order)
    counter = 0
    for mode in modes_of_order:
        counter += 1
        neighbour = strid.find_nearest_neighbour(mode, modes[order-1])

        if (np.isnan(strid.rel_diff_freq(neighbour.f, mode.f))):
            mode.delta_frequency = 1
        else:
            mode.delta_frequency = strid.rel_diff_freq(neighbour.f, mode.f)

        if (np.isnan(np.abs(neighbour.xi - mode.xi))):
            mode.delta_damping = 1
        else:
            mode.delta_damping = np.abs(neighbour.xi - mode.xi)

        if (np.isnan(strid.utils.modal_assurance_criterion(neighbour.v, mode.v))):
            mode.mac = 1
        else:
            mode.delta_mac = strid.utils.modal_assurance_criterion(neighbour.v, mode.v)

        cluster_meat.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])

cluster_meat = np.array(cluster_meat)
# !!! Have to handle modes in the lowest order!!!

/Users/emilneby/PyCharmProjects/TKT4550_Prosjekt/strid/utils.py:728: RuntimeWarning: invalid value encountered in double_scalars
  return self.eigenvalue.imag / np.sqrt(1-self.xi**2)


In [7]:
#print(cluster_meat[:,0])

In [8]:
#Making a 3D scatterplot of frequency, damping and MAC number
fig = plt.figure()
ax = plt.axes(projection='3d')
fg = ax.scatter3D(cluster_meat[:,0], cluster_meat[:,1], cluster_meat[:,2])
ax.set_xlabel("Frequency")
ax.set_ylabel("Damping")
ax.set_zlabel("MAC")

print("Total number of modes = " +str(cluster_meat.shape[0]))

<IPython.core.display.Javascript object>

Total number of modes = 1210


### Task 2.2 -  Using K-means clustering to separate all the poles into two groups

In [9]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(cluster_meat)
labels = kmeans.labels_

In [10]:
#Assign label to each mode
physical_coordinates = []
mathematical_coordinates = []
count = 0
physical_modes_dict = {}
physical_modes_list = []
num_modes = 0
for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))
    physical_modes_in_order = []
    for mode in modes_of_order:
        mode.physical = labels[count]
        if (mode.physical == 0):
            physical_modes_in_order.append(mode)
            physical_modes_list.append(mode)
            physical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        else:
            mathematical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        count += 1
    physical_modes_dict[order] = physical_modes_in_order
    num_modes += len(physical_modes_in_order)

physical_coordinates = np.array(physical_coordinates)
mathematical_coordinates = np.array(mathematical_coordinates)
physical_modes_list = np.array(physical_modes_list)


In [11]:
print(physical_modes_list.shape[0])

1116


In [12]:
%matplotlib nbagg
#Making a 3D scatterplot of frequency, damping and MAC number divided into physical and mathematical clusters
fig = plt.figure()
ax = plt.axes(projection='3d')

fg = ax.scatter3D(physical_coordinates[:,0], physical_coordinates[:,1], physical_coordinates[:,2], c="blue", label= "Physical modes")
ax.scatter3D(mathematical_coordinates[:,0], mathematical_coordinates[:,1], mathematical_coordinates[:,2], c="red", label="Mathematical modes")
ax.legend()
ax.set_xlabel("Frequency")
ax.set_ylabel("Damping")
ax.set_zlabel("MAC")

print("Number of physical modes = " + str(physical_coordinates.shape[0]))
print("Number of mathematical modes = " +str(mathematical_coordinates.shape[0]))

<IPython.core.display.Javascript object>

Number of physical modes = 1116
Number of mathematical modes = 94


In [13]:
%matplotlib nbagg
#Making a new stabilization diagram with the physical modes
stabdiag = strid.StabilizationDiagram()
stabdiag.plot(physical_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>

# Task 3
### Task 3.1 Detecting structural modes by hierachical clustering
Using hierarchical clustering with a signle linkage method with a cut-off-distance of $d_c=0.04$
$$ d_{c_{i,j}} = d \lambda_{i,j} + (1- MAC_{i,j}) $$

In [14]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html

# https://www.youtube.com/watch?v=v7oLMvcxgFY&ab_channel=KindsonTheTechPro

import strid
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering


#Since we want to define our own distance between two poles, we must compute a distance matrix we can feed into the clustering algorithm.


In [15]:
def distance_matrix(modes: np.ndarray) -> np.ndarray:
    """
    Computes the distance matrix between structural modes.
    The distance between two modes i and j is defined as:
    dc_ij = delta_eigenvalues_ij + (1 - MAC_ij)
    Parameters
    ----------
    modes: 1d numpy array with elements of class Mode

    Returns: 2d numpy array that is a distance matrix
    -------

    """
    #Preallocatig matrix to store the distances in
    dist_matrix = np.zeros((modes.shape[0], modes.shape[0]))

    for i in range(0, dist_matrix.shape[0]-1):
        for j in range(0, dist_matrix.shape[1]-1):
            #Computing the distance at each element
            eigen_i = np.sqrt(modes[i].eigenvalue**2)
            eigen_j = np.sqrt(modes[j].eigenvalue**2)
            if i != j:
                dist_matrix[i,j] = (np.abs((eigen_i - eigen_j))/np.max([eigen_i, eigen_j])) + (1 - strid.utils.modal_assurance_criterion(modes[i].v, modes[j].v))

    return dist_matrix

In [16]:
distance = distance_matrix(physical_modes_list)

/var/folders/xg/ddkjb2x96w167hqnv_gjv_v40000gn/T/ipykernel_84295/2650019114.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  dist_matrix[i,j] = (np.abs((eigen_i - eigen_j))/np.max([eigen_i, eigen_j])) + (1 - strid.utils.modal_assurance_criterion(modes[i].v, modes[j].v))


In [17]:
#Check the integrity of the distance matrix
print(np.max(distance))
print(np.mean(distance))
print(np.min(distance))

2.041735419019204
0.9274079150808134
0.0


In [18]:
plt.figure()
plt.title("Dendogram")
dendogram = sch.dendrogram(sch.linkage(distance, method = 'ward'))

<IPython.core.display.Javascript object>

/var/folders/xg/ddkjb2x96w167hqnv_gjv_v40000gn/T/ipykernel_84295/2090417883.py:3: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  dendogram = sch.dendrogram(sch.linkage(distance, method = 'ward'))


In [19]:
#Perform the actual clustering
hc = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='average', distance_threshold=0.04)
#The agglomerative clustering algorithm only finds 1 non-structural mode with single linkage method. Works better so far with 'complete' or 'average'.

In [20]:
y_hc = hc.fit_predict(distance)
print((y_hc))
print(np.max(y_hc))

[125 185 221 ... 107 105 125]
253


In [21]:
hierarchy = y_hc.tolist()
print(len(y_hc))
print(type(hierarchy))
#print(hierarchy)

num_modes_in_hierarchy = np.zeros((np.max(y_hc) + 1, 3))
for i in range(0, len(num_modes_in_hierarchy)):
    num_modes_in_hierarchy[i, 0] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 1] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 2] = i

print((num_modes_in_hierarchy))

1116
<class 'list'>
[[ 22.  22.   0.]
 [ 22.  22.   1.]
 [  6.   6.   2.]
 [  2.   2.   3.]
 [  6.   6.   4.]
 [  2.   2.   5.]
 [  2.   2.   6.]
 [  6.   6.   7.]
 [  6.   6.   8.]
 [  3.   3.   9.]
 [  2.   2.  10.]
 [  2.   2.  11.]
 [  4.   4.  12.]
 [  2.   2.  13.]
 [  5.   5.  14.]
 [  3.   3.  15.]
 [  6.   6.  16.]
 [  6.   6.  17.]
 [  3.   3.  18.]
 [  3.   3.  19.]
 [  4.   4.  20.]
 [  5.   5.  21.]
 [  6.   6.  22.]
 [  3.   3.  23.]
 [  9.   9.  24.]
 [  8.   8.  25.]
 [  3.   3.  26.]
 [  2.   2.  27.]
 [  6.   6.  28.]
 [  6.   6.  29.]
 [  2.   2.  30.]
 [  1.   1.  31.]
 [  3.   3.  32.]
 [  3.   3.  33.]
 [  3.   3.  34.]
 [  2.   2.  35.]
 [  6.   6.  36.]
 [  3.   3.  37.]
 [  2.   2.  38.]
 [  2.   2.  39.]
 [ 36.  36.  40.]
 [  2.   2.  41.]
 [  3.   3.  42.]
 [  2.   2.  43.]
 [  3.   3.  44.]
 [  6.   6.  45.]
 [  3.   3.  46.]
 [ 39.  39.  47.]
 [  2.   2.  48.]
 [  9.   9.  49.]
 [  8.   8.  50.]
 [  4.   4.  51.]
 [  3.   3.  52.]
 [  3.   3.  53.]
 [  1.  

In [22]:
structural_coordinates = []
non_structural_coordinates = []

for i in range(0, len(y_hc)):
    if y_hc[i] == 1:
        structural_coordinates.append(physical_coordinates[i,:])
    else:
        non_structural_coordinates.append(physical_coordinates[i,:])

structural_coordinates = np.array(structural_coordinates)
non_structural_coordinates = np.array(non_structural_coordinates)

In [23]:
#Assign hierarchy to each mode
count = 0
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    for mode in modes_of_order:
        mode.cluster = hierarchy[count]
        count += 1

# Task 4
### Task 4.1

In [24]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(num_modes_in_hierarchy[:,:2])
labels2 = kmeans.labels_

In [25]:
#print(labels2)
print(len(labels2))

254


In [26]:
structural_hierarchies = []

for i in range(0, len(labels2)):
    if(labels2[i] == 1):
        structural_hierarchies.append(i)

print(structural_hierarchies)
#need to create a dictionary with colors to the clusters

[0, 1, 40, 47, 74, 81, 84, 87, 90, 105, 107, 139, 161, 170, 190, 234]


In [27]:
#Assign hierarchy to each mode
structural_modes_dict = {}
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    structural_modes_in_order = []
    for mode in modes_of_order:
        #print(mode.f)
        if mode.cluster in structural_hierarchies:
            structural_modes_in_order.append(mode)

    structural_modes_dict[order] = structural_modes_in_order

In [28]:
%matplotlib nbagg
#Making a new stabilization diagram with the physical modes
stabdiag = strid.StabilizationDiagram()
stabdiag.plot_clusters(structural_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>